In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

# Step 1: Load the dataset
df = pd.read_csv("WineQT.csv")

# Step 2: Drop the 'Id' column (not needed)
df.drop(columns=["Id"], inplace=True)

# Step 3: Separate features (X) and target (y)
X = df.drop(columns=["quality"])  # Features
y = df["quality"]  # Target variable (discrete classes)

# Step 4: Map target labels to a range of [0, num_classes - 1]
unique_classes = y.unique()
num_classes = len(unique_classes)
label_mapping = {label: idx for idx, label in enumerate(sorted(unique_classes))}
y_mapped = y.map(label_mapping)

# Step 5: Perform Z-score normalization (standardization)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert the scaled features back to a DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Step 6: Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_mapped, test_size=0.2, random_state=42)

# Step 7: Define a function to build and train the model
def build_and_train_model(num_neurons=64, learning_rate=0.001, batch_size=32):
    model = Sequential([
        Dense(num_neurons, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
        Dense(num_neurons // 2, activation='relu'),  # Hidden layer
        Dense(num_neurons // 4, activation='relu'),  # Hidden layer
        Dense(num_classes, activation='softmax')  # Output layer
    ])
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_split=0.2, verbose=0)
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred = tf.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Step 8: Test different hyperparameter values
# Hyperparameter 1: Number of Neurons in the Hidden Layers
neurons_list = [32, 64, 128]
print("Testing Number of Neurons:")
for neurons in neurons_list:
    accuracy = build_and_train_model(num_neurons=neurons)
    print(f"Neurons: {neurons}, Accuracy: {accuracy:.4f}")

# Hyperparameter 2: Learning Rate
learning_rates = [0.0001, 0.001, 0.01]
print("\nTesting Learning Rates:")
for lr in learning_rates:
    accuracy = build_and_train_model(learning_rate=lr)
    print(f"Learning Rate: {lr}, Accuracy: {accuracy:.4f}")

# Hyperparameter 3: Batch Size
batch_sizes = [16, 32, 64]
print("\nTesting Batch Sizes:")
for batch_size in batch_sizes:
    accuracy = build_and_train_model(batch_size=batch_size)
    print(f"Batch Size: {batch_size}, Accuracy: {accuracy:.4f}")

Testing Number of Neurons:
8/8 [==============================] - 0s 6ms/step
Neurons: 32, Accuracy: 0.6201
8/8 [==============================] - 0s 4ms/step
Neurons: 64, Accuracy: 0.5852
8/8 [==============================] - 0s 3ms/step
Neurons: 128, Accuracy: 0.5983

Testing Learning Rates:
8/8 [==============================] - 0s 3ms/step
Learning Rate: 0.0001, Accuracy: 0.6114
8/8 [==============================] - 0s 4ms/step
Learning Rate: 0.001, Accuracy: 0.6245
8/8 [==============================] - 0s 3ms/step
Learning Rate: 0.01, Accuracy: 0.6114

Testing Batch Sizes:
8/8 [==============================] - 0s 2ms/step
Batch Size: 16, Accuracy: 0.6245
8/8 [==============================] - 0s 2ms/step
Batch Size: 32, Accuracy: 0.6157
8/8 [==============================] - 0s 4ms/step
Batch Size: 64, Accuracy: 0.5939


### **Answer to Question 9: Hyperparameter Tuning**

To optimize the Neural Network’s performance, we tested **3 hyperparameters** with **3 different values** each:

1. **Number of Neurons in the Hidden Layers**:
   - **32 Neurons**: Accuracy = **62.01%**
   - **64 Neurons**: Accuracy = **58.52%**
   - **128 Neurons**: Accuracy = **59.83%**
   - **Best Value**: **32 Neurons** achieved the highest accuracy.

2. **Learning Rate**:
   - **0.0001**: Accuracy = **61.14%**
   - **0.001**: Accuracy = **62.45%**
   - **0.01**: Accuracy = **61.14%**
   - **Best Value**: **0.001** achieved the highest accuracy.

3. **Batch Size**:
   - **16**: Accuracy = **62.45%**
   - **32**: Accuracy = **61.57%**
   - **64**: Accuracy = **59.39%**
   - **Best Value**: **16** achieved the highest accuracy.

#### **Conclusion**:
- The best hyperparameter combination for this dataset is:
  - **Number of Neurons**: **32**
  - **Learning Rate**: **0.001**
  - **Batch Size**: **16**